- join_output 행 개수와 zip_code_datetime_join의 행 개수를 비교 ㄲ
- 추가로 agg_data 행 개수도 찍어보기

In [ ]:
base_query = """
WITH base_data AS (
    # nyc_taxi 테이블의 좌표가 gis에 있는 geo값이 포함되어 있는 row만 출력하여 zip_code 정보를 얻어내는 테이블
    SELECT nyc_taxi.*, gis.* EXCEPT (zip_code_geom)
    FROM (
        SELECT *
        FROM `bigquery-public-data.new_york_taxi_trips.tlc_yellow_trips_2015`
        WHERE 
            EXTRACT(MONTH from pickup_datetime) = 1
            and pickup_latitude BETWEEN -90 AND 90
        ) AS nyc_taxi
    JOIN (
        SELECT zip_code, state_code, state_name, city, county, zip_code_geom
        FROM `bigquery-public-data.geo_us_boundaries.zip_codes`
        WHERE state_code='NY'
        ) AS gis 
    ON ST_CONTAINS(zip_code_geom, st_geogpoint(pickup_longitude, pickup_latitude))
)
, distinct_datetime AS (
    # base_data 에서 hour 이후 값을 제거하고 유니크한 시간단위의 pickup data의 정보를 얻어내는 테이블
    # Datetime만 distinct해서 가져옴(비어있지 않을것이라 가정)
    SELECT distinct DATETIME_TRUNC(pickup_datetime, hour) as pickup_hour
    FROM base_data
)
, distinct_zip_code AS (
    # base_data 에서 zip_code의 정보를 얻어내는 테이블
    # zip_code만 distinct해서 가져옴(이외의 zip_code는 나오지 않을것이라 가정)
    SELECT distinct zip_code
    FROM base_data
)
, zip_code_datetime_join AS (
    # zip_code, pickup_hour의 정보를 가진 두 테이블을 join하여 여러 단위의 날짜 데이터를 얻는 테이블
    SELECT
        *,
        EXTRACT(MONTH FROM pickup_hour) AS month,
        EXTRACT(DAY FROM pickup_hour) AS day,
        CAST(FORMAT_DATETIME('%u', pickup_hour) AS INT64) -1 AS weekday,
        EXTRACT(HOUR FROM pickup_hour) AS hour,
        CASE WHEN CAST(FORMAT_DATETIME('%u', pickup_hour) AS INT64) IN (6, 7) THEN 1 ELSE 0 END AS is_weekend
    FROM distinct_zip_code  
    CROSS JOIN distinct_datetime
)
, agg_data AS (
    # base_data 에서 zip_code별 시간 단위의 픽업 수에 대한 정보 테이블
    SELECT 
        zip_code,
        DATETIME_TRUNC(pickup_datetime, hour) as pickup_hour,
        COUNT(*) AS cnt
    FROM base_data 
    GROUP BY zip_code, pickup_hour
)
, join_output AS (
    # zip_code별 여러 시간 단위에 대한 수요값 테이블
    # zip_code, datetime 데이터에 수요값을 붙이고 없다면 0처리
    SELECT 
        zip_code_datetime.*, 
        IFNULL(agg_data.cnt, 0) AS cnt
    FROM zip_code_datetime_join AS zip_code_datetime
    LEFT JOIN agg_data
    ON zip_code_datetime.zip_code = agg_data.zip_code and zip_code_datetime.pickup_hour = agg_data.pickup_hour
)

SELECT
    *,
    LAG(cnt, 1) OVER(PARTITION BY zip_code ORDER BY pickup_hour) AS lag_1h_cnt,
    LAG(cnt, 24) OVER(PARTITION BY zip_code ORDER BY pickup_hour) AS lag_1d_cnt,
    LAG(cnt, 168) OVER(PARTITION BY zip_code ORDER BY pickup_hour) AS lag_7d_cnt,
    LAG(cnt, 336) OVER(PARTITION BY zip_code ORDER BY pickup_hour) AS lag_14d_cnt,
    ROUND(AVG(cnt) OVER(PARTITION BY zip_code ORDER BY pickup_hour ROWS BETWEEN 168 PRECEDING AND 1 PRECEDING), 2) AS avg_7d_cnt,
    ROUND(AVG(cnt) OVER(PARTITION BY zip_code ORDER BY pickup_hour ROWS BETWEEN 336 PRECEDING AND 1 PRECEDING), 2) AS avg_14d_cnt,
    CAST(STDDEV(cnt) OVER(PARTITION BY zip_code ORDER BY pickup_hour ROWS BETWEEN 168 PRECEDING AND 1 PRECEDING) AS INT64) AS std_7d_cnt,
    CAST(STDDEV(cnt) OVER(PARTITION BY zip_code ORDER BY pickup_hour ROWS BETWEEN 336 PRECEDING AND 1 PRECEDING) AS INT64) AS std_14d_cnt
FROM join_output
order by zip_code, pickup_hour
"""